In [ ]:
# tags to identify this iteration when submitted
# example: codex_tags = {'env': 'dev', 'region': 'USA', 'product_category': 'A'}

codex_tags = {
}

from codex_widget_factory import utils
results_json=[]

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
HistoricScreenFilters = """
## Below codestring is used to create filters to show different regions data in  product historic screen.

import pandas as pd
import json
from itertools import chain


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'GlobalLandTemperaturesByCity_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        
        #print(input_df)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_response_filters(current_filter_params, df, default_values_selected, all_filters, multi_select_filters, extra_filters={}):
    logger.info("Preparing filter dictionary")
    # Usage
    # -----
    # >>> filter_df = pd.DataFrame(columns=[....])    # Optional operation
    # >>> filter_df = final_ADS.groupby(......)       # Optional operation
    # >>> default_values_selected = {}    # The default value to be selected for a filter, provide filter_name, filter_values
    # >>> all_option_filters = []         # Filters with an All option
    # >>> multi_select_filters = []       # Filters with an multi_select option
    # >>> more_filters = {}               # Extra filters, provide filter_names, filter_options
    # >>> final_dict_out = get_response_filters(current_filter_params, filter_df, default_values_selected, all_option_filters, multi_select_filters, more_filters)
    # >>> dynamic_outputs = json.dumps(final_dict_out)
    # Returns
    # -------
    # A dict object containing the filters JSON structure

    filters = list(df.columns)
    default_values_possible = {}
    for item in filters:
        default_possible = list(df[item].unique())
        if item in all_filters:
            default_possible = list(chain(['All'], default_possible))
        default_values_possible[item] = default_possible
    if extra_filters:
        filters.extend(list(extra_filters.keys()))
        default_values_possible.update(extra_filters)
    if current_filter_params:
        selected_filters = current_filter_params["selected"]
        # current_filter = current_filter_params["current_filter"]
        # current_index = filters.index(current_filter)
        select_df = df.copy()
    final_dict = {}
    iter_value = 0
    data_values = []
    default_values = {}
    for item in filters:
        filter_dict = {}
        filter_dict["widget_filter_index"] = int(iter_value)
        filter_dict["widget_filter_function"] = False
        filter_dict["widget_filter_function_parameter"] = False
        filter_dict["widget_filter_hierarchy_key"] = False
        filter_dict["widget_filter_isall"] = True if item in all_filters else False
        filter_dict["widget_filter_multiselect"] = True if item in multi_select_filters else False
        filter_dict["widget_tag_key"] = str(item)
        filter_dict["widget_tag_label"] = str(item)
        filter_dict["widget_tag_input_type"] = "select",
        filter_dict["widget_filter_dynamic"] = True
        if current_filter_params:
            if item in df.columns:
                possible_values = list(select_df[item].unique())
                item_default_value = selected_filters[item]
                if item in all_filters:
                    possible_values = list(chain(['All'], possible_values))
                if item in multi_select_filters:
                    for value in selected_filters[item]:
                        if value not in possible_values:
                            if possible_values[0] == "All":
                                item_default_value = possible_values
                            else:
                                item_default_value = [possible_values[0]]
                else:
                    if selected_filters[item] not in possible_values:
                        item_default_value = possible_values[0]
                filter_dict["widget_tag_value"] = possible_values
                if item in multi_select_filters:
                    if 'All' not in item_default_value and selected_filters[item]:
                        select_df = select_df[select_df[item].isin(
                            item_default_value)]
                else:
                    if selected_filters[item] != 'All':
                        select_df = select_df[select_df[item]
                                              == item_default_value]
            else:
                filter_dict["widget_tag_value"] = extra_filters[item]
        else:
            filter_dict["widget_tag_value"] = default_values_possible[item]
            item_default_value = default_values_selected[item]
        data_values.append(filter_dict)
        default_values[item] = item_default_value
        iter_value = iter_value + 1
    final_dict["dataValues"] = data_values
    final_dict["defaultValues"] = default_values
    logger.info("Successfully prepared filter dictionary")
    return final_dict


def prepare_filter_json():
    logger.info(f"Preparing json for Filters in Historical Screen")
    # Preapre Filter json for Region in the Historical View Screen.
    dframe = fiile_read("cost-of-living_v2_I0869")
    dframe = dframe.groupby(['country']).sum().reset_index()
    filter_dframe = dframe[['country']]
    default_values_selected = {"country": 'India'}
    all_filters = []
    multi_select_filters = []
    current_filter_params = {"selected": default_values_selected}
    final_dict_out = get_response_filters(
        current_filter_params, filter_dframe, default_values_selected, all_filters, multi_select_filters)
    logger.info(f"Successful prepared json for Filters in Historical Screen")
    return json.dumps(final_dict_out)


dynamic_outputs = prepare_filter_json()
"""
#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
historicScreenGridTable = """
# Below codestring is used to display the grid table that consists of historic sales done over time on each product.

import pandas as pd
import json


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'GlobalLandTemperaturesByCity_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        
        #print(input_df)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_filter_table(dframe, selected_filters):
    logger.info("Applying screen filters on the grid table dframe.")
    select_df = dframe.copy()
    for item in list(selected_filters):
        if isinstance(selected_filters[item], list):
            if 'All' not in selected_filters[item] and selected_filters[item]:
                select_df = select_df[select_df[item].isin(
                    selected_filters[item])]
        else:
            if selected_filters[item] != 'All':
                select_df = select_df[select_df[item]
                                      == selected_filters[item]]
    logger.info("Successfully applied screen filters on the grid table dframe.")
    return select_df


def generate_dynamic_table(dframe, name='Sales', grid_options={"tableSize": "small", "tableMaxHeight": "80vh", "quickSearch":True}, group_headers=[], grid="auto"):
    logger.info("Generate dynamic Grid table json from dframe")
    table_dict = {}
    table_props = {}
    table_dict.update({"grid": grid, "type": "tabularForm",
                      "noGutterBottom": True, 'name': name})
    values_dict = dframe.dropna(axis=1).to_dict("records")
    table_dict.update({"value": values_dict})
    col_def_list = []
    for col in list(dframe.columns):
        col_def_dict = {}
        col_def_dict.update({"headerName": col, "field": col})
        col_def_list.append(col_def_dict)
    table_props["groupHeaders"] = group_headers
    table_props["coldef"] = col_def_list
    table_props["gridOptions"] = grid_options
    table_dict.update({"tableprops": table_props})
    logger.info("Successfully generated dynamic Grid table json from dframe")
    return table_dict


def build_grid_table_json():
    logger.info("Preparing grid table json for Historical Screen")
    form_config = {}
    dframe = fiile_read("cost-of-living_v2_I0869")
    selected_filters = {"country": 'India'}
    dframe = get_filter_table(dframe, selected_filters)
    form_config['fields'] = [generate_dynamic_table(dframe)]
    grid_table_json = {}
    grid_table_json['form_config'] = form_config
    logger.info("Successfully prepared grid table json for Historical Screen")
    return grid_table_json


grid_table_json = build_grid_table_json()
dynamic_outputs = json.dumps(grid_table_json)

"""
#END CUSTOM CODE done till here

In [ ]:
BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
historicScreenActionGen = """
# Below codestring is used to create action generator that triggers action handler  to display breadcrumbs 
# to show total units of product sale over time.

import pandas as pd
import json


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def get_action_generator():
    logger.info(f"Preparing action generator json for Historical screen")
    action_generator = {
        "actions": [{
            "action_type": "get_screen_breadcrumbs",
            "component_type": "text_list",
            "params": {
                "fetch_on_load": True
            },
            "position": {
                "portal": "screen_top_left"
            }
        }]
    }

    logger.info(
        f"Successfully prepared action generator json for Historical screen")
    return action_generator


res = get_action_generator()
dynamic_outputs = json.dumps(res)

"""
#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets

#historicScreenActionHandler = """
# Below codestring is used to generate json for breadcrumbs to display total number units of sales done over time.

import pandas as pd
import json


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'GlobalLandTemperaturesByCity_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        
        #print(input_df)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_quantity_breadcrumbs():
    logger.info(
        f"Preparing action handler json for order quantity in historical screen")
    dframe = fiile_read("cost-of-living_v2_I0869")
    total_quantity = dframe['OrderQuantity'].sum()
    actions = {
        "list": [
            {
                "text": "number of people: ",
            },
            {
                "text": str(total_quantity),
                "color": "contrast",
                "style": {
                    "fontWeight": 600,
                }
            }
        ]
    }
    logger.info(
        f"Successfully prepared action handler json for order quantity in historical screen")
    return actions


# action_type = "get_screen_breadcrumbs"


if action_type == "get_screen_breadcrumbs":
    res = get_quantity_breadcrumbs()

dynamic_outputs = json.dumps(res)

"""

#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
#productReturn_quantity = """
# Below codestring is used to display the nuumber of product unit returned in a year.

import pandas as pd
import json


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'GlobalLandTemperaturesByCity_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        
        #print(input_df)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_productReturnQuantity():
    logger.info("Calculating Quantity of product returned")
    dframe = read_dataset("ProductReturns.csv") #*test#
    productReturnQuantity = dframe['ReturnQuantity'].sum()
    metric = {}
    metric['title'] = "Quantity of Products Returned"
    metric['value'] = str(int(productReturnQuantity)) + ' units'
    logger.info("Successfully calculated Quantity of product returned")
    return metric


kpi_json = get_productReturnQuantity()
dynamic_outputs = json.dumps(kpi_json)
"""
#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
#productReturn_UniqueProduct = """
# Below codestring is used to display the nuumber of uniue product types returned in a year.

import pandas as pd
import json


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def read_dataset(filename):
    # Read dataset from the github.
    logger.info(f"Read dataset file: {filename}")
    try:
        dataset_url = f"https://raw.githubusercontent.com/saipraneeth4/codx_dataset/main/dataset/{filename}"
        dframe = pd.read_csv(dataset_url)
        return dframe
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {dataset_url}"
                    f"Error Info is  {error_msg}")


def get_uniqueProductReturned():
    logger.info("Calculating number of unique products returned")
    dframe = read_dataset("ProductReturns.csv")
    uniqueProductQuantity = len(dframe['ProductName'].unique())
    metric = {}
    metric['title'] = "No of Unique Products Returned"
    metric['value'] = str(int(uniqueProductQuantity)) + ' unique Products'
    logger.info("Successfully calculated number of unique products returned")
    return metric


kpi_json = get_uniqueProductReturned()
dynamic_outputs = json.dumps(kpi_json)
"""
#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
# productReturns_costOfProductReturned = """
# Below codestring is used to display the total cost of product that was returned in a year.

import pandas as pd
import json


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def read_dataset(filename):
    # Read dataset from the github.
    logger.info(f"Read dataset file: {filename}")
    try:
        dataset_url = f"https://raw.githubusercontent.com/saipraneeth4/codx_dataset/main/dataset/{filename}"
        dframe = pd.read_csv(dataset_url)
        return dframe
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {dataset_url}"
                    f"Error Info is  {error_msg}")


def get_costOfProductReturned():
    logger.info("Calculating Totalcost of products returned")
    dframe = read_dataset("ProductReturns.csv")
    costOfProdRetuned = dframe['ProductPrice'].sum()
    metric = {}
    metric['title'] = "No of Unique Products Returned"
    metric['value'] = str(int(costOfProdRetuned)) + ' $'
    logger.info("Successfully calculated Totalcost of products returned")
    return metric


kpi_json = get_costOfProductReturned()
dynamic_outputs = json.dumps(kpi_json)
"""
#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
# orderDetailQuantity = """
# Below codestring is used to perform detailed analysis on quantity of sales done over time. 

import plotly.express as px
import pandas as pd
import json
import plotly.io as io


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def read_dataset(filename):
    # Read dataset from the github.
    logger.info(f"Read dataset file: {filename}")
    try:
        dataset_url = f"https://raw.githubusercontent.com/saipraneeth4/codx_dataset/main/dataset/{filename}"
        dframe = pd.read_csv(dataset_url)
        return dframe
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {dataset_url}"
                    f"Error Info is  {error_msg}")


def getGraph(dframe, filters):
    logger.info(
        "Preparing bar graph json to understand products sales over time")
    for item in filters:
        if 'All' in filters[item]:
            continue
        elif isinstance(filters[item], list):
            dframe = dframe[dframe[item].isin(filters[item])]
        else:
            dframe = dframe[dframe[item] == filters[item]]
    fig = px.bar(dframe, x='OrderDate', y='OrderQuantity', color='ProductName')
    # fig.show()
    logger.info(
        "Successfully prepared bar graph json to understand products sales over time")
    return io.to_json(fig)


# selected_filters = {"Region": 'Australia'}
dframe = read_dataset("ProductSales.csv")
dynamic_outputs = getGraph(dframe, selected_filters)

"""
#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
# oderDetailProduct = """
# Below codestring is used to perform detailed analysis of Product stock available over time. 

import plotly.express as px
import pandas as pd
import json
import plotly.io as io


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def read_dataset(filename):
    # Read dataset from the github.
    logger.info(f"Read dataset file: {filename}")
    try:
        dataset_url = f"https://raw.githubusercontent.com/saipraneeth4/codx_dataset/main/dataset/{filename}"
        dframe = pd.read_csv(dataset_url)
        return dframe
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {dataset_url}"
                    f"Error Info is  {error_msg}")


def getGraph(dframe, filters):
    logger.info(
        "Preparing bar graph json to understand products in stock available over time")
    for item in filters:
        if 'All' in filters[item]:
            continue
        elif isinstance(filters[item], list):
            dframe = dframe[dframe[item].isin(filters[item])]
        else:
            dframe = dframe[dframe[item] == filters[item]]
    fig = px.bar(dframe, x='StockDate', y='OrderQuantity', color='ProductName')
    # fig.show()
    logger.info(
        "Successfully prepared bar graph json to understand products in stock available over time")
    return io.to_json(fig)


selected_filters = {"Region": 'Australia'}
dframe = read_dataset("ProductSales.csv")
dynamic_outputs = getGraph(dframe, selected_filters)
"""
#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
#productReturns_gridTable = """
# Below codestring is used to view the product returns details.

import pandas as pd
import json


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def read_dataset(filename):
    # Read dataset from the github.
    logger.info(f"Read dataset file: {filename}")
    try:
        dataset_url = f"https://raw.githubusercontent.com/saipraneeth4/codx_dataset/main/dataset/{filename}"
        dframe = pd.read_csv(dataset_url)
        return dframe
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {dataset_url}"
                    f"Error Info is  {error_msg}")


def generate_dynamic_table(dframe, name='Sales', grid_options={"tableSize": "small", "tableMaxHeight": "80vh", "quickSearch":True}, group_headers=[], grid="auto"):
    logger.info("Generate dynamic Grid table json from dframe")
    table_dict = {}
    table_props = {}
    table_dict.update({"grid": grid, "type": "tabularForm",
                      "noGutterBottom": True, 'name': name})
    values_dict = dframe.dropna(axis=1).to_dict("records")
    table_dict.update({"value": values_dict})
    col_def_list = []
    for col in list(dframe.columns):
        col_def_dict = {}
        col_def_dict.update({"headerName": col, "field": col})
        col_def_list.append(col_def_dict)
    table_props["groupHeaders"] = group_headers
    table_props["coldef"] = col_def_list
    table_props["gridOptions"] = grid_options
    table_dict.update({"tableprops": table_props})
    logger.info("Successfully generated dynamic Grid table json from dframe")
    return table_dict


def build_grid_table_json():
    logger.info("Preparing grid table json for Product Returns Screen")
    form_config = {}
    dframe = read_dataset("ProductReturns.csv")
    form_config['fields'] = [generate_dynamic_table(dframe)]
    grid_table_json = {}
    grid_table_json['form_config'] = form_config
    logger.info("Successfully prepared grid table json for Product Returns Screen")
    return grid_table_json


grid_table_json = build_grid_table_json()
dynamic_outputs = json.dumps(grid_table_json)
"""
#END CUSTOM CODE

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
#orderDetailFilter = """
## Below codestring is used to create filters to show different regions data in  Order Detail Analysis screen.

import pandas as pd
import json
from itertools import chain


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def read_dataset(filename):
    # Read dataset from the github.
    logger.info(f"Read dataset file: {filename}")
    try:
        dataset_url = f"https://raw.githubusercontent.com/saipraneeth4/codx_dataset/main/dataset/{filename}"
        dframe = pd.read_csv(dataset_url)
        return dframe
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {dataset_url}"
                    f"Error Info is  {error_msg}")


def get_response_filters(current_filter_params, df, default_values_selected, all_filters, multi_select_filters, extra_filters={}):
    logger.info("Preparing filter dictionary")
    # Usage
    # -----
    # >>> filter_df = pd.DataFrame(columns=[....])    # Optional operation
    # >>> filter_df = final_ADS.groupby(......)       # Optional operation
    # >>> default_values_selected = {}    # The default value to be selected for a filter, provide filter_name, filter_values
    # >>> all_option_filters = []         # Filters with an All option
    # >>> multi_select_filters = []       # Filters with an multi_select option
    # >>> more_filters = {}               # Extra filters, provide filter_names, filter_options
    # >>> final_dict_out = get_response_filters(current_filter_params, filter_df, default_values_selected, all_option_filters, multi_select_filters, more_filters)
    # >>> dynamic_outputs = json.dumps(final_dict_out)
    # Returns
    # -------
    # A dict object containing the filters JSON structure

    filters = list(df.columns)
    default_values_possible = {}
    for item in filters:
        default_possible = list(df[item].unique())
        if item in all_filters:
            default_possible = list(chain(['All'], default_possible))
        default_values_possible[item] = default_possible
    if extra_filters:
        filters.extend(list(extra_filters.keys()))
        default_values_possible.update(extra_filters)
    if current_filter_params:
        selected_filters = current_filter_params["selected"]
        # current_filter = current_filter_params["current_filter"]
        # current_index = filters.index(current_filter)
        select_df = df.copy()
    final_dict = {}
    iter_value = 0
    data_values = []
    default_values = {}
    for item in filters:
        filter_dict = {}
        filter_dict["widget_filter_index"] = int(iter_value)
        filter_dict["widget_filter_function"] = False
        filter_dict["widget_filter_function_parameter"] = False
        filter_dict["widget_filter_hierarchy_key"] = False
        filter_dict["widget_filter_isall"] = True if item in all_filters else False
        filter_dict["widget_filter_multiselect"] = True if item in multi_select_filters else False
        filter_dict["widget_tag_key"] = str(item)
        filter_dict["widget_tag_label"] = str(item)
        filter_dict["widget_tag_input_type"] = "select",
        filter_dict["widget_filter_dynamic"] = True
        if current_filter_params:
            if item in df.columns:
                possible_values = list(select_df[item].unique())
                item_default_value = selected_filters[item]
                if item in all_filters:
                    possible_values = list(chain(['All'], possible_values))
                if item in multi_select_filters:
                    for value in selected_filters[item]:
                        if value not in possible_values:
                            if possible_values[0] == "All":
                                item_default_value = possible_values
                            else:
                                item_default_value = [possible_values[0]]
                else:
                    if selected_filters[item] not in possible_values:
                        item_default_value = possible_values[0]
                filter_dict["widget_tag_value"] = possible_values
                if item in multi_select_filters:
                    if 'All' not in item_default_value and selected_filters[item]:
                        select_df = select_df[select_df[item].isin(
                            item_default_value)]
                else:
                    if selected_filters[item] != 'All':
                        select_df = select_df[select_df[item]
                                              == item_default_value]
            else:
                filter_dict["widget_tag_value"] = extra_filters[item]
        else:
            filter_dict["widget_tag_value"] = default_values_possible[item]
            item_default_value = default_values_selected[item]
        data_values.append(filter_dict)
        default_values[item] = item_default_value
        iter_value = iter_value + 1
    final_dict["dataValues"] = data_values
    final_dict["defaultValues"] = default_values
    logger.info("Successfully prepared filter dictionary")
    return final_dict


def prepare_filter_json():
    logger.info(f"Preparing json for Filters in Order Detail Screen")
    # Preapre Filter json for Region in the Order Detail View Screen.
    dframe = read_dataset("ProductSales.csv")
    dframe = dframe.groupby(['Region']).sum().reset_index()
    filter_dframe = dframe[['Region']]
    default_values_selected = {"Region": 'Australia'}
    all_filters = []
    multi_select_filters = []
    # current_filter_params = {"selected": default_values_selected}
    final_dict_out = get_response_filters(
        current_filter_params, filter_dframe, default_values_selected, all_filters, multi_select_filters)
    logger.info(f"Successful prepared json for Filters in Order Detail Screen")
    return json.dumps(final_dict_out)


dynamic_outputs = prepare_filter_json()

"""
#END CUSTOM CODE